Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/08 12:59:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/08 12:59:24 WARN ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


In [2]:


from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql import SparkSession
from operator import add
from pyspark.sql.functions import *
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.11:7077") \
        .appName("test")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",1)\
        .config("spark.cores.max",2)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()
        #.config("spark.executor.memory", "1gb")\  ##?????????
        #.config("spark.dynamicAllocation.minExecutors", 1)
        
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

reddit = spark_context.textFile("hdfs://192.168.2.11:9000/user/ubuntu/redditcomments/RC_2005-12")



In [3]:
reddit_df = spark_session.read.json(reddit)
reddit_df.printSchema()

root
 |-- author: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created_utc: long (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- edited: boolean (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- retrieved_on: long (nullable = true)
 |-- score: long (nullable = true)
 |-- stickied: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- ups: long (nullable = true)



In [4]:
reddit_df = reddit_df.select("body", "controversiality", "subreddit", "subreddit_id", "created_utc")
reddit_df.show(1)

+--------------------+----------------+----------+------------+-----------+
|                body|controversiality| subreddit|subreddit_id|created_utc|
+--------------------+----------------+----------+------------+-----------+
|A look at Vietnam...|               0|reddit.com|        t5_6| 1134365188|
+--------------------+----------------+----------+------------+-----------+
only showing top 1 row



In [10]:

contro_subreddit = reddit_df.select("controversiality", "subreddit_id", "subreddit")

avg_controversial = contro_subreddit.groupBy("subreddit_id").agg(avg("controversiality"), count("subreddit"))
    
avg_controversial_sorted = avg_controversial.orderBy("avg(controversiality)",ascending=False)
print(avg_controversial_sorted.show(10))



+------------+---------------------+----------------+
|subreddit_id|avg(controversiality)|count(subreddit)|
+------------+---------------------+----------------+
|        t5_6| 0.046511627906976744|            1075|
+------------+---------------------+----------------+

None


In [24]:
test = reddit_df.select("controversiality").where(col("controversiality") == 1)
print(test.show())

+----------------+
|controversiality|
+----------------+
|               1|
|               1|
|               1|
|               1|
|               1|
|               1|
|               1|
|               1|
|               1|
|               1|
|               1|
|               1|
|               1|
|               1|
|               1|
|               1|
|               1|
|               1|
|               1|
|               1|
+----------------+
only showing top 20 rows

None


In [25]:
print(avg_controversial_sorted.show(10))


+------------+---------------------+----------------+
|subreddit_id|avg(controversiality)|count(subreddit)|
+------------+---------------------+----------------+
|        t5_6| 0.046511627906976744|            1075|
+------------+---------------------+----------------+

None
